In [1]:
Zx.<x> = ZZ[]
W, N, Q = 5, 7, 256

In [2]:
def convolution(f,g):
    return (f*g) % (x^N-1)

In [3]:
def randomsecret():
    f = list(randrange(3)-1 for j in range(N))
    return Zx(f)

In [4]:
f = randomsecret()
f

-x^6 - x^4 + x^3 + x - 1

In [5]:
def balancedmod(f, Q):
    g = list(((f[i]+Q//2) % Q)-Q//2 for i in range(N))
    return Zx(g)

In [6]:
u = 314-159*x
balancedmod(u, 200)

41*x - 86

In [7]:
def randomweight():
    R = randrange
    assert W <= N
    s = N*[0]
    for j in range(W):
        while True:
            r = R(N)
            if not s[r]:
                break
        s[r] = 1-2*R(2)
    return Zx(s)
                

In [8]:
randomweight()

-x^6 - x^5 - x^3 + x^2 - x

In [9]:
def invertmodprime(f, p):
    Fp = Integers(p)
    Fpx = Zx.change_ring(Fp)
    T = Fpx.quotient(x^N-1)
    return Zx(lift(1/T(f)))
def invertmodpowerof2(f, Q):
    assert Q.is_power_of(2)
    g = invertmodprime(f,2)
    M = balancedmod
    conv = convolution
    while True:
        r = M(conv(g,f), Q)
        if r == 1:
            return g
        g = M(conv(g, 2-r), Q)
        

In [10]:
g = invertmodpowerof2(f, Q)
g

47*x^6 - 36*x^5 + 126*x^4 - 58*x^3 - 54*x^2 + 61*x - 87

In [11]:
balancedmod(convolution(f,g), Q)

1

In [18]:
def keypair():
    while True:
        try:
            a = randomweight()
            a3 = invertmodprime(a, 3)
            aQ = invertmodpowerof2(a, Q)
            e = randomsecret()
            G = balancedmod(3 * convolution(e, aQ), Q)
            GQ = invertmodpowerof2(G, Q)
            secretkey = a,a3,GQ
            return G,secretkey
        except:
            pass

In [36]:
def encrypt(bd,G):
    b,d = bd
    bG = convolution(b,G)
    C = balancedmod(bG+d, Q)
    return C
def decrypt(C, secretkey):
    M = balancedmod
    conv = convolution
    a,a3,GQ = secretkey
    u = M(conv(C,a), Q)
    d = M(conv(u,a3), 3)
    b = M(conv(C-d,GQ), Q)
    return b,d

In [34]:
G, secretkey = keypair()
a,a3,GQ = secretkey
b = randomweight()
d = randomsecret()
C = encrypt((b,d), G)
C

-29*x^6 - 66*x^5 - 19*x^4 + 62*x^3 - 111*x^2 + 36*x - 112

In [39]:
#G,a,convolution(a,G),balancedmod(convolution(a,G),Q)
#decrypt(C,secretkey)
decrypt(C,secretkey) == (b,d)
# W, N, Q = 5, 7, 256

True